In [1]:
%run create_tables.py

In [2]:
%run etl.py

INSERT INTO songplays
                           (start_time,user_id,level,song_id,artist_id,session_id,location,user_agent)
                           SELECT DISTINCT TIMESTAMP 'epoch' + (e.ts / 1000) * INTERVAL '1 second' as start_time, 
                           e.userId, e.level, 
                           s.song_id, a.artist_id, 
                           e.sessionId, e.location, e.userAgent 
                           FROM stagging_events e LEFT JOIN (
                               SELECT * FROM (
                                   SELECT title, song_id, row_number () OVER
                                   (PARTITION BY title) AS rownum FROM songs) groups
                                   WHERE groups.rownum = 1) s
                           ON (e.song=s.title) LEFT JOIN (
                               SELECT * FROM (
                                    SELECT name, artist_id, ROW_NUMBER () OVER
                                    (PARTITION BY name) AS rownum FROM artists